# Import Library

In [11]:
import os
import tabula
import numpy as np
import pandas as pd
from pandas import DataFrame
import PyPDF2 as pyPdf

# Instruction: Change the location of the 2 folder

In [12]:
#update path where the downloded 'Data Mining' were stored
path = r"C:\Users\woru6001\Desktop" 

# Read All the PDF files in the folder

In [13]:
arr = []
for file in os.listdir(path + r"\Data Mining\Python Code\Daily Stock Price"):
    if file.endswith(".pdf"):
        arr.append(file)

filenames = [f for f in arr]  

output = pd.DataFrame([])
final = pd.DataFrame([])
filenames

['securities_equities_010419.pdf',
 'securities_equities_020419.pdf',
 'securities_equities_030419.pdf',
 'securities_equities_040419.pdf',
 'securities_equities_050419.pdf',
 'securities_equities_080419.pdf',
 'securities_equities_090419.pdf',
 'securities_equities_100419.pdf',
 'securities_equities_110419.pdf',
 'securities_equities_120419.pdf',
 'securities_equities_150419.pdf',
 'securities_equities_160419.pdf',
 'securities_equities_170419.pdf',
 'securities_equities_180419.pdf',
 'securities_equities_190419.pdf',
 'securities_equities_250319.pdf',
 'securities_equities_260319.pdf',
 'securities_equities_270319.pdf',
 'securities_equities_280319.pdf',
 'securities_equities_290319.pdf']

# Read and append tables from selected pages 

In [14]:
for data in filenames:
    reader = pyPdf.PdfFileReader(open(data,mode='rb') )
    n = reader.getNumPages() 
    for i in range(4,n):
        df1 = tabula.read_pdf(data,pages=i,sort=True)
        output=output.append(df1)
    final = final.append(output)

of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.
 [frame.py:6692]


In [15]:
test=final.copy()
test.head()
test.shape

(409302, 11)

# Data Cleaning

In [16]:
test['Stock Name']=test['Stock Name'].fillna(test['Unnamed: 2'])
test = test.drop("Unnamed: 2", axis=1)
#test=test.dropna()
test.shape

(409302, 10)

In [17]:
#Retaining Leading Zero in StockCode
test.Stock = test.Stock.apply('="{}"'.format)

#Remove Duplicate and NA 
test = test.drop_duplicates()
test=test.dropna()
print(test.shape)

(37180, 10)


In [18]:
test.tail()

,Closing,Cur.,Date,High,Low,Opening,Stock,Stock Name,Value Traded,Volume
54,1.140,MYR,29/03/2019,1.140,1.100,1.100,"=""9687""",IDEAL,"5,614","5,100"
55,0.355,MYR,29/03/2019,0.370,0.355,0.370,"=""9687WB""",IDEAL-WB,"12,934","35,000"
56,0.970,MYR,29/03/2019,0.970,0.960,0.960,"=""9695""",PLS,"1,087,677","1,131,900"
57,0.255,MYR,29/03/2019,0.265,0.225,0.225,"=""9717""",SYCAL,"201,359","815,400"
58,0.650,MYR,29/03/2019,0.660,0.650,0.660,"=""9741""",ROHAS,"32,550","50,000"


# Rearrange and rename Columns 

In [19]:
test2 = test[['Date', 'Stock', 'Stock Name', 'Cur.','Opening', 'Closing','High','Low','Value Traded','Volume']]
test2.columns = ['Date', 'Stock Code', 'Stock Name', 'Currency','Opening', 'Closing','High','Low','Value Traded','Volume Traded']
test2.head()

,Date,Stock Code,Stock Name,Currency,Opening,Closing,High,Low,Value Traded,Volume Traded
3,01/04/2019,"=""0001""",SCOMNET,MYR,0.795,0.790,0.810,0.785,"500,654","630,500"
4,01/04/2019,"=""0002""",KOTRA,MYR,1.690,1.690,1.690,1.690,"15,717","9,300"
5,01/04/2019,"=""0005""",UCREST,MYR,0.260,0.255,0.265,0.255,"1,999,327","7,728,700"
6,01/04/2019,"=""0006""",PINEAPP,MYR,0.000,0.320,0.000,0.000,0,0
7,01/04/2019,"=""0007""",PUC,MYR,0.095,0.095,0.095,0.095,"795,359","8,372,200"


In [20]:
test2.head()

,Date,Stock Code,Stock Name,Currency,Opening,Closing,High,Low,Value Traded,Volume Traded
3,01/04/2019,"=""0001""",SCOMNET,MYR,0.795,0.790,0.810,0.785,"500,654","630,500"
4,01/04/2019,"=""0002""",KOTRA,MYR,1.690,1.690,1.690,1.690,"15,717","9,300"
5,01/04/2019,"=""0005""",UCREST,MYR,0.260,0.255,0.265,0.255,"1,999,327","7,728,700"
6,01/04/2019,"=""0006""",PINEAPP,MYR,0.000,0.320,0.000,0.000,0,0
7,01/04/2019,"=""0007""",PUC,MYR,0.095,0.095,0.095,0.095,"795,359","8,372,200"


In [13]:
#test2.to_csv('Stock_Data.csv',index=False)

# Remove Warrant Stock

In [21]:
test3= test2[~test2['Stock Code'].str.contains("A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z",na=False)]
test3.shape

(26223, 10)

In [22]:
test3= test3[~test3['Stock Name'].str.contains("-",na=False)]
test3.shape

(19747, 10)

# Extract, Import Market and Sector Information


In [23]:
data=path + r'\Data Mining\Python Code\Daily Stock Price\Market Sector Information\Securities_Equities_Year_End_Closing_Prices_2018.pdf'
output = pd.DataFrame(tabula.read_pdf(data,pages=1))
output=output.dropna()
output2= pd.DataFrame([])
reader = pyPdf.PdfFileReader(open(data,mode='rb') )
n = reader.getNumPages() 
for i in range(2,n):
    df1 = tabula.read_pdf(data,pages=i,sort=True,pandas_options={'header': None},multiple_tables=True)
    output2=output2.append(df1)
output2.head()

,0,1,2,3,4,5
0,0066,VSOLAR,VSOLAR GROUP BERHAD,ACE MARKET,TECHNOLOGY,0.120
1,0068,ASDION,ASDION BHD,ACE MARKET,TECHNOLOGY,0.185
2,0068WB,ASDION-WB,ASDION BERHAD - WARRANTS 2014/2019,ACE MARKET,TECHNOLOGY,0.010
3,0069,VIVOCOM,VIVOCOM INTL HOLDINGS BERHAD,ACE MARKET,TECHNOLOGY,0.020
4,0069WC,VIVOCOM-WC,VIVOCOM INTL HOLDINGS BERHAD - WARRANTS C 2015...,ACE MARKET,TECHNOLOGY,0.005


In [24]:
output2 = output2.rename(columns = {0: "Stock", 
                                1:"Stock Name",
                                2: "Stock Long Name",
                               3: "Board",
                               4:"Sector",
                               5:"Closing"})
output3=output.append(output2)
print(output3.shape)
output3.Stock=output3.Stock.apply('="{}"'.format)
output3.tail()

(1843, 6)


,Stock,Stock Name,Stock Long Name,Board,Sector,Closing
61,"=""9539""",MUH,MULTI-USAGE HOLDINGS BHD,MAIN MARKET,PROPERTY,0.490
62,"=""9571""",MITRA,MITRAJAYA HOLDINGS BHD,MAIN MARKET,CONSTRUCTION,0.280
63,"=""9571WD""",MITRA-WD,MITRAJAYA HOLDINGS BERHAD-WARRANTS D 15/20,MAIN MARKET,CONSTRUCTION,0.030
64,"=""9571WE""",MITRA-WE,MITRAJAYA HOLDINGS BERHAD - WARRANTS E 2018/2023,MAIN MARKET,CONSTRUCTION,0.055
65,"=""9598""",PTARAS,PINTARAS JAYA BHD,MAIN MARKET,CONSTRUCTION,2.230


In [25]:
sector = output3[['Stock', 'Board', 'Sector']]
sector.columns = ['Stock Code','Board', 'Sector']
sector.head()

,Stock Code,Board,Sector
2,"=""0001""",ACE MARKET,INDUSTRIAL PRODUCTS & SERVICES
3,"=""0002""",MAIN MARKET,HEALTH CARE
4,"=""0005""",ACE MARKET,TECHNOLOGY
5,"=""0006""",ACE MARKET,TECHNOLOGY
6,"=""0007""",ACE MARKET,TELECOMMUNICATIONS & MEDIA


# vlookup sector and market information

In [26]:
consolidated=pd.merge(test3, sector, on='Stock Code', how='left')

In [27]:
consolidated.head()

,Date,Stock Code,Stock Name,Currency,Opening,Closing,High,Low,Value Traded,Volume Traded,Board,Sector
0,01/04/2019,"=""0001""",SCOMNET,MYR,0.795,0.790,0.810,0.785,"500,654","630,500",ACE MARKET,INDUSTRIAL PRODUCTS & SERVICES
1,01/04/2019,"=""0002""",KOTRA,MYR,1.690,1.690,1.690,1.690,"15,717","9,300",MAIN MARKET,HEALTH CARE
2,01/04/2019,"=""0005""",UCREST,MYR,0.260,0.255,0.265,0.255,"1,999,327","7,728,700",ACE MARKET,TECHNOLOGY
3,01/04/2019,"=""0006""",PINEAPP,MYR,0.000,0.320,0.000,0.000,0,0,ACE MARKET,TECHNOLOGY
4,01/04/2019,"=""0007""",PUC,MYR,0.095,0.095,0.095,0.095,"795,359","8,372,200",ACE MARKET,TELECOMMUNICATIONS & MEDIA


# Converting data types

In [28]:
consolidated.dtypes

Date             object
Stock Code       object
Stock Name       object
Currency         object
Opening          object
Closing          object
High             object
Low              object
Value Traded     object
Volume Traded    object
Board            object
Sector           object
dtype: object

In [29]:
#Converting str to integer
consolidated['Value Traded'] = consolidated['Value Traded'].str.replace(",","").astype(int)
consolidated['Volume Traded'] = consolidated['Volume Traded'].str.replace(",","").astype(int)
consolidated['Opening'] = pd.to_numeric(consolidated.Opening, errors='coerce')
consolidated['Closing'] = pd.to_numeric(consolidated.Closing, errors='coerce')
consolidated['High'] = pd.to_numeric(consolidated.High, errors='coerce')
consolidated['Low'] = pd.to_numeric(consolidated.Low, errors='coerce')

In [30]:
consolidated.dtypes

Date              object
Stock Code        object
Stock Name        object
Currency          object
Opening          float64
Closing          float64
High             float64
Low              float64
Value Traded       int32
Volume Traded      int32
Board             object
Sector            object
dtype: object

In [31]:
#Forming Dependant Variables
consolidated['test']=consolidated['Closing']-consolidated['Opening']

consolidated.loc[consolidated.test > 0, 'StockPriceChange'] = 'Increase'
consolidated.loc[consolidated.test < 0, 'StockPriceChange'] = 'Decrease'
consolidated.loc[consolidated.test == 0, 'StockPriceChange'] = 'Remain'
del consolidated['test']

In [32]:
consolidated.head()

,Date,Stock Code,Stock Name,Currency,Opening,Closing,High,Low,Value Traded,Volume Traded,Board,Sector,StockPriceChange
0,01/04/2019,"=""0001""",SCOMNET,MYR,0.795,0.790,0.810,0.785,500654,630500,ACE MARKET,INDUSTRIAL PRODUCTS & SERVICES,Decrease
1,01/04/2019,"=""0002""",KOTRA,MYR,1.690,1.690,1.690,1.690,15717,9300,MAIN MARKET,HEALTH CARE,Remain
2,01/04/2019,"=""0005""",UCREST,MYR,0.260,0.255,0.265,0.255,1999327,7728700,ACE MARKET,TECHNOLOGY,Decrease
3,01/04/2019,"=""0006""",PINEAPP,MYR,0.000,0.320,0.000,0.000,0,0,ACE MARKET,TECHNOLOGY,Increase
4,01/04/2019,"=""0007""",PUC,MYR,0.095,0.095,0.095,0.095,795359,8372200,ACE MARKET,TELECOMMUNICATIONS & MEDIA,Remain


In [33]:
consolidated['High']=consolidated['High'].fillna(0)
consolidated['Low']=consolidated['Low'].fillna(0)

# Export Consolidated Clean Data

In [34]:
consolidated.to_csv(path + r'\Data Mining\Python Code\Output\Stock_Data_Final_filtered.csv',index=False)
consolidated.to_excel (path + r'\Data Mining\Python Code\Output\Stock_Data_Final_filtered.xlsx', index = None, header=True)

# Run Covariance Analysis

In [35]:
#import back the clean dataset and select only stock closing price information
df = pd.read_csv(path + r'\Data Mining\Python Code\Output\Stock_Data_Final_filtered.csv')
df1=df[['Date','Stock Name','Closing']]
df1.head(n=8)

,Date,Stock Name,Closing
0,01/04/2019,SCOMNET,0.790
1,01/04/2019,KOTRA,1.690
2,01/04/2019,UCREST,0.255
3,01/04/2019,PINEAPP,0.320
4,01/04/2019,PUC,0.095
5,01/04/2019,WILLOW,0.455
6,01/04/2019,IRIS,0.155
7,01/04/2019,BTECH,0.335


In [36]:
df2=df1.pivot_table(index='Stock Name', columns='Date',values='Closing')
df2.reset_index(inplace=True)

In [37]:
df2 = df2.replace(np.NaN,0)
print(df2.isnull().sum())

Date
Stock Name    0
01/04/2019    0
02/04/2019    0
03/04/2019    0
04/04/2019    0
05/04/2019    0
08/04/2019    0
09/04/2019    0
10/04/2019    0
11/04/2019    0
12/04/2019    0
15/04/2019    0
16/04/2019    0
17/04/2019    0
18/04/2019    0
19/04/2019    0
25/03/2019    0
26/03/2019    0
27/03/2019    0
28/03/2019    0
29/03/2019    0
dtype: int64


In [38]:
df2.head(n=8)

Date,Stock Name,01/04/2019,02/04/2019,03/04/2019,04/04/2019,05/04/2019,08/04/2019,09/04/2019,10/04/2019,11/04/2019,...,15/04/2019,16/04/2019,17/04/2019,18/04/2019,19/04/2019,25/03/2019,26/03/2019,27/03/2019,28/03/2019,29/03/2019
0,3A,0.930,0.930,0.940,0.940,0.935,0.935,0.935,0.945,0.935,...,0.930,0.925,0.940,0.920,0.925,0.950,0.98,0.970,0.935,0.935
1,AASIA,0.150,0.150,0.150,0.150,0.150,0.150,0.150,0.150,0.150,...,0.150,0.165,0.145,0.145,0.160,0.160,0.16,0.160,0.160,0.150
2,AAX,0.240,0.245,0.245,0.245,0.240,0.235,0.230,0.230,0.230,...,0.220,0.225,0.230,0.230,0.230,0.245,0.25,0.245,0.245,0.245
3,ABLEGRP,0.075,0.080,0.075,0.075,0.080,0.080,0.085,0.085,0.085,...,0.080,0.085,0.080,0.085,0.085,0.070,0.07,0.070,0.070,0.070
4,ABMB,4.010,4.000,4.080,4.120,4.120,4.100,4.090,4.090,4.080,...,4.030,4.020,4.000,4.000,4.000,4.140,4.14,4.070,4.040,4.090
5,ACME,0.250,0.250,0.250,0.260,0.260,0.260,0.260,0.260,0.260,...,0.300,0.300,0.300,0.300,0.300,0.250,0.25,0.250,0.250,0.250
6,ACOSTEC,0.420,0.420,0.420,0.415,0.420,0.415,0.420,0.420,0.415,...,0.415,0.415,0.415,0.405,0.405,0.430,0.42,0.415,0.425,0.420
7,ADVCON,0.440,0.440,0.435,0.430,0.430,0.445,0.470,0.475,0.460,...,0.435,0.420,0.410,0.425,0.425,0.420,0.43,0.425,0.410,0.425


In [39]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# Separating out the features
x = df2.iloc[:,1:].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

In [40]:
print (x.shape)

(1022, 20)


In [41]:
#running PCA components to reduce the dimentionality of the dataset
#running PCA =3
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3'])
pca.explained_variance_ratio_
sum(pca.explained_variance_ratio_)*100

99.98456145293945

In [42]:
#Running PCA =2
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
pca.explained_variance_ratio_
sum(pca.explained_variance_ratio_)*100

99.96684448870957

# Since 99.97% of the variance can be explained by 2 PCA, hence we will not add in the 3rd PCA 

In [43]:
Companyname=df2['Stock Name'].tolist()

In [44]:
#obtain the covariance of the reduced dataset  
y = np.array(principalDf)
covariance = pd.DataFrame(np.cov(y),columns=Companyname,index=Companyname)
variance = np.var(y, axis=1, ddof=1)
print(covariance)

                  3A     AASIA       AAX   ABLEGRP       ABMB      ACME  \
3A          0.114232  0.253657  0.238972  0.266762  -0.437752  0.232787   
AASIA       0.253657  0.563257  0.530649  0.592358  -0.972048  0.516913   
AAX         0.238972  0.530649  0.499928  0.558064  -0.915774  0.486988   
ABLEGRP     0.266762  0.592358  0.558064  0.622962  -1.022269  0.543619   
ABMB       -0.437752 -0.972048 -0.915774 -1.022269   1.677525 -0.892070   
ACME        0.232787  0.516913  0.486988  0.543619  -0.892070  0.474382   
ACOSTEC     0.206763  0.459126  0.432546  0.482847  -0.792343  0.421350   
ADVCON      0.203292  0.451419  0.425285  0.474741  -0.779042  0.414276   
ADVENTA     0.217554  0.483088  0.455121  0.508047  -0.833696  0.443340   
ADVPKG     -0.048064 -0.106727 -0.100549 -0.112241   0.184186 -0.097946   
AEM         0.261868  0.581489  0.547825  0.611532  -1.003512  0.533645   
AEMULUS     0.238334  0.529232  0.498593  0.556575  -0.913329  0.485688   
AEON        0.022334  0.0

In [45]:
#export the covariance result 
pd.DataFrame(covariance).to_csv(path + r'\Data Mining\Python Code\Output\Covariance.csv')
